In [178]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [151]:
train=pd.read_excel('D:\iSmile Technologies\Side Projects\Flight Fare prediction\Flight_Ticket_Participant_Datasets\Data_train.xlsx')
test=pd.read_excel('D:\iSmile Technologies\Side Projects\Flight Fare prediction\Flight_Ticket_Participant_Datasets\Test_set.xlsx')

In [141]:
submission=pd.read_excel('D:\iSmile Technologies\Side Projects\Flight Fare prediction\Flight_Ticket_Participant_Datasets\Sample_submission.xlsx')

In [7]:
train.shape,test.shape

((10683, 11), (2671, 10))

In [6]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [48]:
train.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [49]:
train.loc[train['Price'].idxmax()]

Airline            Jet Airways Business
Date_of_Journey              01/03/2019
Source                         Banglore
Destination                   New Delhi
Route                   BLR → BOM → DEL
Dep_Time                          05:45
Arrival_Time                      11:25
Duration                         5h 40m
Total_Stops                      1 stop
Additional_Info          Business class
Price                             79512
Name: 2924, dtype: object

In [152]:
train['Journey_day']=pd.to_datetime(train.Date_of_Journey, format="%d/%m/%Y").dt.day
train['Journey_month']=pd.to_datetime(train.Date_of_Journey, format="%d/%m/%Y").dt.month
train['Dep_hour']=pd.to_datetime(train.Dep_Time).dt.hour
train['Dep_minute']=pd.to_datetime(train.Dep_Time).dt.minute
train['Arrival_hour']=pd.to_datetime(train.Arrival_Time).dt.hour
train['Arrival_minute']=pd.to_datetime(train.Arrival_Time).dt.minute

test['Journey_day']=pd.to_datetime(test.Date_of_Journey, format="%d/%m/%Y").dt.day
test['Journey_month']=pd.to_datetime(test.Date_of_Journey, format="%d/%m/%Y").dt.month
test['Dep_hour']=pd.to_datetime(test.Dep_Time).dt.hour
test['Dep_minute']=pd.to_datetime(test.Dep_Time).dt.minute
test['Arrival_hour']=pd.to_datetime(test.Arrival_Time).dt.hour
test['Arrival_minute']=pd.to_datetime(test.Arrival_Time).dt.minute

In [62]:
print(train)

           Airline Date_of_Journey    Source Destination  \
0           IndiGo      24/03/2019  Banglore   New Delhi   
1        Air India       1/05/2019   Kolkata    Banglore   
2      Jet Airways       9/06/2019     Delhi      Cochin   
3           IndiGo      12/05/2019   Kolkata    Banglore   
4           IndiGo      01/03/2019  Banglore   New Delhi   
...            ...             ...       ...         ...   
10678     Air Asia       9/04/2019   Kolkata    Banglore   
10679    Air India      27/04/2019   Kolkata    Banglore   
10680  Jet Airways      27/04/2019  Banglore       Delhi   
10681      Vistara      01/03/2019  Banglore   New Delhi   
10682    Air India       9/05/2019     Delhi      Cochin   

                       Route Dep_Time  Arrival_Time Duration Total_Stops  \
0                  BLR → DEL    22:20  01:10 22 Mar   2h 50m    non-stop   
1      CCU → IXR → BBI → BLR    05:50         13:15   7h 25m     2 stops   
2      DEL → LKO → BOM → COK    09:25  04:25 10 Jun

In [153]:
train['Route']=train['Route'].fillna('No Route')
train['Total_Stops']=train['Total_Stops'].fillna('non-stop')
test['Route']=test['Route'].fillna('No Route')
test['Total_Stops']=test['Total_Stops'].fillna('non-stop')

In [74]:
train.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [131]:
test.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Journey_day        0
Journey_month      0
Dep_hour           0
Dep_minute         0
Arrival_hour       0
Arrival_minute     0
dtype: int64

In [154]:
dur=list(train['Duration'])
duration_hour=[]
duration_min=[]
for i in dur:
    temp=i.split(' ')
    if len(temp)==2:
        m=temp[1]
        m=m[:-1]
    h=temp[0]
    h=h[:-1]
    duration_hour.append(int(h))
    duration_min.append(int(m))
train['duration_hour']=duration_hour
train['duraiton_minute']=duration_min

In [155]:
dur=list(test['Duration'])
duration_hour=[]
duration_min=[]
for i in dur:
    temp=i.split(' ')
    if len(temp)==2:
        m=temp[1]
        m=m[:-1]
    h=temp[0]
    h=h[:-1]
    duration_hour.append(int(h))
    duration_min.append(int(m))
test['duration_hour']=duration_hour
test['duraiton_minute']=duration_min

In [156]:
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
train['Destination']=enc.fit_transform(train['Destination'])
train['Airline']=enc.fit_transform(train['Airline'])
train['Source']=enc.fit_transform(train['Source'])
train['Route']=enc.fit_transform(train['Route'])
train['Additional_Info']=enc.fit_transform(train['Additional_Info'])
train['Total_Stops']=enc.fit_transform(train['Total_Stops'])

test['Destination']=enc.fit_transform(test['Destination'])
test['Airline']=enc.fit_transform(test['Airline'])
test['Source']=enc.fit_transform(test['Source'])
test['Route']=enc.fit_transform(test['Route'])
test['Additional_Info']=enc.fit_transform(test['Additional_Info'])
test['Total_Stops']=enc.fit_transform(test['Total_Stops'])

In [80]:
!pip install xgboost

In [157]:
X=train.drop(['Price','Date_of_Journey','Dep_Time','Arrival_Time','Duration'],axis=1)
Y=train['Price']

In [158]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

# Using Xgboost Algorithm

In [164]:
t1=datetime.now()
np.random.seed(10)
model = XGBRegressor(
      max_depth=7,
      n_estimators=500,
      min_child_weight=300, 
      colsample_bytree=0.8, 
      subsample=0.8, 
      gamma = 0.005,
      eta=0.1,    
      seed=42)
model.fit(
    X_train, 
    Y_train, 
    verbose=10,
    )
t2=datetime.now()
print(t2-t1)

0:00:02.488611


In [165]:
y_pred_tr=model.predict(X_train)
y_pred_test=model.predict(X_test)
tr_r2=r2_score(Y_train,y_pred_tr)
test_r2=r2_score(Y_test,y_pred_test)
print(tr_r2,test_r2)

0.8779758546833795 0.8218629662994954


In [168]:
test=test.drop(['Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration'],axis=1)

In [142]:
submission.head()

,Price
0,15998
1,16612
2,25572
3,25778
4,16934


In [150]:
final_sub=model.predict(test)
submission['Price']=final_sub
submission.to_csv('D:\iSmile Technologies\Side Projects\Flight Fare prediction\Flight_Ticket_Participant_Datasets\Final_submission_2.csv')

# Using Decision Tree

In [177]:
max_depth=[5,10,15,20]
min_sample_split=[25,50,100,150]
for i in max_depth:
    for j in min_sample_split:
        model=DecisionTreeRegressor(max_depth=i,min_samples_split=j)
        model.fit(X_train,Y_train)
        y_pred_tr=model.predict(X_train)
        y_pred_test=model.predict(X_test)
        tr_r2=r2_score(Y_train,y_pred_tr)
        test_r2=r2_score(Y_test,y_pred_test)
        print('Max Depth:{} Min sample split:{} {} {} {}'.format(i,j,tr_r2,test_r2,abs(tr_r2-test_r2)))

Max Depth:5 Min sample split:25 0.677454578421339 0.5861778682847757 0.09127671013656335
Max Depth:5 Min sample split:50 0.676054803920312 0.591330797690749 0.084724006229563
Max Depth:5 Min sample split:100 0.676054803920312 0.5908179407633901 0.08523686315692192
Max Depth:5 Min sample split:150 0.676054803920312 0.5908179407633901 0.08523686315692192
Max Depth:10 Min sample split:25 0.8627689839377208 0.7908702343760168 0.07189874956170406
Max Depth:10 Min sample split:50 0.8516870113005015 0.7764726155928333 0.07521439570766819
Max Depth:10 Min sample split:100 0.8346959564431109 0.7612442123829258 0.07345174406018506
Max Depth:10 Min sample split:150 0.8188373215085027 0.741150651060763 0.07768667044773969
Max Depth:15 Min sample split:25 0.9032595857632943 0.826563112981253 0.07669647278204139
Max Depth:15 Min sample split:50 0.8779964363517127 0.8119025624447277 0.06609387390698496
Max Depth:15 Min sample split:100 0.8471828326403554 0.7773966330238915 0.06978619961646393
Max Dep

In [175]:
model=DecisionTreeRegressor(max_depth=15,min_samples_split=500)
model.fit(X_train,Y_train)

DecisionTreeRegressor(max_depth=15, min_samples_split=500)

In [176]:
final_sub=model.predict(test)
submission['Price']=final_sub
submission.to_csv('D:\iSmile Technologies\Side Projects\Flight Fare prediction\Flight_Ticket_Participant_Datasets\Final_submission_DT.csv')

# Using Random Forest Algorithm

In [183]:
max_depth=[10,15,20]
min_sample_split=[25,50,100,150]
n_estimators=[50,100,150,200,500,1000]
for i in n_estimators:
    for j in min_sample_split:
        for k in max_depth:
            model=RandomForestRegressor(n_estimators=k,max_depth=i,min_samples_split=j)
            model.fit(X_train,Y_train)
            y_pred_tr=model.predict(X_train)
            y_pred_test=model.predict(X_test)
            tr_r2=r2_score(Y_train,y_pred_tr)
            test_r2=r2_score(Y_test,y_pred_test)
            print('n_estimators:{} Max Depth:{} Min sample split:{} {} {} {}'.format(i,k,j,tr_r2,test_r2,abs(tr_r2-test_r2)))
    print('\n')

n_estimators:50 Max Depth:10 Min sample split:25 0.9049563876747492 0.8507405037375235 0.05421588393722565
n_estimators:50 Max Depth:15 Min sample split:25 0.9131813128364483 0.854066434942674 0.05911487789377434
n_estimators:50 Max Depth:20 Min sample split:25 0.9101808617800304 0.8484354550364699 0.06174540674356055
n_estimators:50 Max Depth:10 Min sample split:50 0.8774974777022553 0.821085052331516 0.056412425370739316
n_estimators:50 Max Depth:15 Min sample split:50 0.8795854534212546 0.8186330069645282 0.06095244645672637
n_estimators:50 Max Depth:20 Min sample split:50 0.8850397737392713 0.8261819251912423 0.058857848548029
n_estimators:50 Max Depth:10 Min sample split:100 0.8466506188909544 0.7673295777596532 0.07932104113130123
n_estimators:50 Max Depth:15 Min sample split:100 0.8470328384819977 0.7714071554610448 0.07562568302095296
n_estimators:50 Max Depth:20 Min sample split:100 0.8468728258030475 0.7742171988178276 0.07265562698521988
n_estimators:50 Max Depth:10 Min samp

In [184]:
model=RandomForestRegressor(n_estimators=200,max_depth=15,min_samples_split=25)
model.fit(X_train,Y_train)

RandomForestRegressor(max_depth=15, min_samples_split=25, n_estimators=200)

In [185]:
final_sub=model.predict(test)
submission['Price']=final_sub
submission.to_csv('D:\iSmile Technologies\Side Projects\Flight Fare prediction\Flight_Ticket_Participant_Datasets\Final_submission_randomforest.csv')